# Scratch

This was originally included in the main analysis notebook. I took it out because the notebook was getting too large.

### Simple simulation

Let's simulate the measurement very simply. We can start with the "correct" $\Sigma$ at the reconstruction point (the one we reconstructed using all the measurements). We'll transport this to each wire-scanner using the transfer matrices. We can add some error to the measured moments and try the reconstruction method. 

In [ ]:
def scatter_hist(x, y, ax, ax_marg_x, ax_marg_y, joint_kws=None, marginal_kws=None):
    if joint_kws is None:
        joint_kws = dict()
    if marginal_kws is None:
        marginal_kws = dict()        
    if 'range' in joint_kws:
        xrange, yrange = joint_kws['range']
    else:
        xrange = yrange = None
    if 'bins' in joint_kws:
        bins = joint_kws['bins']
    else:
        heights, edges, patches = ax_marg_x.hist(x, bins='auto', range=xrange, **marginal_kws)
        for patch in patches:
            patch.set_visible(False)
        bins = len(heights)
        joint_kws['bins'] = bins
    ax_marg_x.hist(x, range=xrange, bins=bins, **marginal_kws)
    ax_marg_y.hist(y, range=yrange, bins=bins, orientation='horizontal', **marginal_kws)
    ax.hist2d(x, y, **joint_kws)
    return ax

def create_grid(fig, gridspec, row, col):
    ax_joint = fig.add_subplot(gridspec[row, col])
    ax_marg_x = fig.add_subplot(gridspec[row - 1, col])
    ax_marg_y = fig.add_subplot(gridspec[row, col + 1])
    for ax in [ax_marg_x, ax_marg_y]:
        ax.set_xticks([])
        ax.set_yticks([])
        for side in ['top', 'bottom', 'left', 'right']:
            ax.spines[side].set_visible(False)
    return ax_joint, ax_marg_x, ax_marg_y

def emittances_joint_hist(emittances, lims=((10, 40), (0, 20))):
    fig = plt.figure(figsize=(10, 4))
    h = 1.5
    gridspec = fig.add_gridspec(2, 5, width_ratios=(7, h, 2.5, 7, h), height_ratios=(h, 7),
                                left=0.1, right=0.9, bottom=0.1, top=0.9,
                                wspace=0, hspace=0)
    ax1, ax1_marg_x, ax1_marg_y = create_grid(fig, gridspec, 1, 0)
    ax2, ax2_marg_x, ax2_marg_y = create_grid(fig, gridspec, 1, 3)

    joint_kws = dict(cmap='fire_r', range=lims, bins=75)
    marginal_kws = dict(histtype='step', color='black')

    scatter_hist(emittances[:, 0], emittances[:, 1], ax1, ax1_marg_x, ax1_marg_y, joint_kws, marginal_kws)
    scatter_hist(emittances[:, 2], emittances[:, 3], ax2, ax2_marg_x, ax2_marg_y, joint_kws, marginal_kws)
    ax1_marg_x.set_xlim(lims[0])
    ax1_marg_y.set_ylim(lims[1])
    ax2_marg_x.set_xlim(lims[0])
    ax2_marg_y.set_ylim(lims[1])
    ax1.set_xlabel(r'$\varepsilon_x$ [mm mrad]')
    ax1.set_ylabel(r'$\varepsilon_y$ [mm mrad]')
    ax2.set_xlabel(r'$\varepsilon_1$ [mm mrad]')
    ax2.set_ylabel(r'$\varepsilon_2$ [mm mrad]')
    ax1_marg_x.set_title(r'Apparent emittances ($\varepsilon_x$, $\varepsilon_y$)')
    ax2_marg_x.set_title(r'Intrinsic emittances ($\varepsilon_1$, $\varepsilon_2$)')      
    return ax1, ax1_marg_x, ax1_marg_y, ax2, ax1_marg_y, ax1_marg_y

In [ ]:
def get_moments(Sigma0, tmats, f=None):
    """Return [<xx>, <yy>, <xy>] at each wire-scanner.
    
    `f` is the rms fractional error added to the virtually measured moments. 
    """
    moments = []
    for M in tmats:            
        Sigma = np.linalg.multi_dot([M, Sigma0, M.T])
        sig_xx = Sigma[0, 0]
        sig_yy = Sigma[2, 2]
        sig_xy = Sigma[0, 2]
        sig_uu = 0.5 * (2 * sig_xy + sig_xx + sig_yy)
        if f:
            sig_xx *= (1.0 + np.random.normal(scale=f))**2
            sig_yy *= (1.0 + np.random.normal(scale=f))**2
            sig_uu *= (1.0 + np.random.normal(scale=f))**2
        sig_xy = 0.5 * (2 * sig_uu - sig_xx - sig_yy)
        moments.append([sig_xx, sig_yy, sig_xy])
    return moments

In [ ]:
def solve(tmats, moments, method='llsq'):
    Axx, Ayy, Axy = [], [], []
    bxx, byy, bxy = [], [], []
    for M, (sig_xx, sig_yy, sig_xy) in zip(tmats, moments):            
        Axx.append([M[0, 0]**2, M[0, 1]**2, 2*M[0, 0]*M[0, 1]])
        Ayy.append([M[2, 2]**2, M[2, 3]**2, 2*M[2, 2]*M[2, 3]])
        Axy.append([M[0, 0]*M[2, 2],  M[0, 1]*M[2, 2],  M[0, 0]*M[2, 3],  M[0, 1]*M[2, 3]])
        bxx.append(sig_xx)
        byy.append(sig_yy)
        bxy.append(sig_xy)
        
    sig_11, sig_22, sig_12 = opt.lsq_linear(Axx, bxx).x
    sig_33, sig_44, sig_34 = opt.lsq_linear(Ayy, byy).x        
    
    if method == 'llsq':
        sig_13, sig_23, sig_14, sig_24 = opt.lsq_linear(Axy, bxy).x
        Sigma = to_mat([sig_11, sig_22, sig_12, sig_33, sig_44, sig_34, sig_13, sig_23, sig_14, sig_24])
        return Sigma
    else:
        raise ValueError("`method` must be in {'llsq'}")

In [ ]:
def run_trials(Sigma0, tmats, n_trials, method='llsq', f=None, pbar=False):
    emittances, n_fail = [], 0
    for _ in (trange(n_trials) if pbar else range(n_trials)):
        moments = get_moments(Sigma0, tmats, f)
        Sigma = solve(tmats, moments, method)
        if not utils.is_positive_definite(Sigma):
            n_fail += 1
            continue
        if np.linalg.det(Sigma) < 0:
            n_fail += 1
            continue
        eps_x, eps_y, eps_1, eps_2 = ba.emittances(Sigma)
        if (eps_1 * eps_2 > eps_x * eps_y):
            n_fail += 1
            continue
        emittances.append(ba.emittances(Sigma))
    fail_rate = n_fail / n_trials
    return fail_rate, np.array(emittances)

In [ ]:
f = 0.02
n_trials = 1000
method = 'llsq'

fail_rates, emittances_list = [], []
for meas_index in range(n_meas):
    tmats = [tmats_dict[ws_id][meas_index] for ws_id in ws_ids]
    fail_rate, emittances = run_trials(Sigma_all_meas, tmats, n_trials, method, f, pbar=True)
    fail_rates.append(fail_rate)
    emittances_list.append(emittances)

In [ ]:
fig, ax = pplt.subplots(figsize=(3, 2))
ax.plot(fail_rates, color='black', marker='.')
ax.format(xlabel='Measurement index', ylabel='Fail rate', 
          ylim=(0, 1), xtickminor=False, xticks=range(len(fail_rates)))

In [ ]:
# line_kws = dict(color='white', lw=0.1)
# hist_kws = dict(cmap='viridis', range=((0, 60), (0, 60)), bins=50)
# fig, axes = plt.subplots(ncols=2, figsize=(6, 3), sharex=True, sharey=True, constrained_layout=True)
# plt.close()

# def update(meas_index):
#     fail_rate = fail_rates[meas_index]
#     emittances = emittances_list[meas_index]
#     for j, ax in zip([0, 2], axes):
#         ax.clear()
#         ax.hist2d(emittances[:, j], emittances[:, j + 1], **hist_kws)
#     axes[0].axvline(eps_x_all_meas, **line_kws)
#     axes[0].axhline(eps_y_all_meas, **line_kws)
#     axes[1].axvline(eps_1_all_meas, **line_kws)
#     axes[1].axhline(eps_2_all_meas, **line_kws)
#     axes[0].set_xlabel(r'$\varepsilon_x$ [mm mrad]')
#     axes[0].set_ylabel(r'$\varepsilon_y$ [mm mrad]')
#     axes[1].set_xlabel(r'$\varepsilon_1$ [mm mrad]')
#     axes[1].set_ylabel(r'$\varepsilon_2$ [mm mrad]')
#     axes[0].set_title('Apparent emittances')
#     axes[1].set_title('Intrinsic emittances');
#     axes[1].annotate('f = {}'.format(f), xy=(0.05, 0.92), xycoords='axes fraction', 
#                      color='white', fontsize='small')
#     axes[1].annotate('fail rate = {:.3f}'.format(fail_rate), xy=(0.05, 0.87), xycoords='axes fraction', 
#                      color='white', fontsize='small')
#     axes[1].annotate('meas index = {}'.format(meas_index), xy=(0.05, 0.82), xycoords='axes fraction', 
#                      color='white', fontsize='small')
    
# animation.FuncAnimation(fig, update, frames=n_meas)

Look at one measurement index as the measurement error is scaled.

In [ ]:
meas_index = 2
n_trials = 1000
rms_frac_errs = np.linspace(0.0, 0.05, 6)

tmats = [tmats_dict[ws_id][meas_index] for ws_id in ws_ids]
emittances_list, fail_rates = [], []
for f in rms_frac_errs:
    fail_rate, emittances = run_trials(Sigma_all_meas, tmats, n_trials, method, f, pbar=True)
    fail_rates.append(fail_rate)
    emittances_list.append(emittances)

In [ ]:
fig, ax = pplt.subplots(figsize=(3, 2))
ax.plot(rms_frac_errs, fail_rates, marker='.', color='k')
ax.format(xlabel='RMS fractional error in measured moments',
          ylabel='Fail rate', grid=True,
          title='meas_index = {}'.format(meas_index),
          ylim=(0, 1))
plt.savefig('_output/failrate_vs_meas_error_{}'.format(meas_index), facecolor='white')

In [ ]:
@gif.frame
def plot_errors(f, fail_rate, emittances):
    lims = ((15, 60), (0, 40))
    ax1, ax1_marg_x, ax1_marg_y, ax2, ax1_marg_y, ax1_marg_y = emittances_joint_hist(emittances, lims=lims)
    ax2.annotate('fail rate = {:.2f}'.format(fail_rate), 
             xy=(0.03, 0.93), xycoords='axes fraction', color='white')  
    ax2.annotate(r'rms frac err = {:.0f}%'.format(100 * dsig), 
             xy=(0.03, 0.87), xycoords='axes fraction', color='white') 
    line_kws = dict(color='white', lw=0.25, alpha=0.5)
    ax1.axvline(eps_x, **line_kws)
    ax1.axhline(eps_y, **line_kws)
    ax2.axvline(eps_1, **line_kws)
    ax2.axhline(eps_2, **line_kws)
    figname = '_output/results_f{:.0f}%.png'.format(100*f)
    plt.savefig(figname, facecolor='white', dpi=250)
    plt.show()
    
frames = []
for dsig, fail_rate, emittances in zip(rms_frac_errs, fail_rates, emittances_list):
    frame = plot_errors(dsig, fail_rate, emittances)
    frames.append(frame)

## Stability conditions

Let "individual reconstruction" refer to the reconstruction with only four wire-scanners at fixed optics. The "badness" of an individual reconstruction seems to depend on the optics. In fact, in some cases it seems to vary smoothly with the measurement index. 

This has apparently been studied before. Below, we investigate the stability conditions derived in this previous work.

In [ ]:
def cond(A):
    m, n = A.shape
    if m != n:
        Ainv = np.matmul(np.linalg.inv(np.matmul(A.T, A)), A.T)
    else:
        Ainv = np.linalg.inv(A)
    return np.linalg.norm(A) * np.linalg.norm(Ainv)

In [ ]:
f = 0.02
n_trials = 1000
method = 'llsq'

fail_rates, condition_numbers, emittances_list = [], [], []
for meas_index in range(n_meas):
    tmats = [tmats_dict[ws_id][meas_index] for ws_id in ws_ids]
    fail_rate, emittances = run_trials(Sigma_all_meas, tmats, n_trials, method, f, pbar=True)
    Axx, Ayy, Axy, bxx, byy, bxy = form_coeff_target_arrays(meas_index)
    fail_rates.append(fail_rate)
    condition_numbers.append(cond(Axy))
    emittances_list.append(emittances)

In [ ]:
fig, ax1 = pplt.subplots(figsize=(3, 2))
colors = ['black', 'red8']
ax1.plot(fail_rates, color=colors[0], marker='.')
ax1.set_ylabel('Fail rate', color=colors[0])
ax1.tick_params(axis='y', labelcolor=colors[0])
ax2 = ax1.twinx()
ax2.plot(np.log10(condition_numbers), color=colors[1], marker='.')
ax2.set_ylabel(r'log$_{10}$ Condition number', color=colors[1])
ax2.tick_params(axis='y', labelcolor=colors[1])
ax1.format(xlabel='Measurement index', xtickminor=False, xticks=range(len(fail_rates)), ylim=(0, 1))
plt.savefig('_output/fail_rate_and_cond_number.png', facecolor='white', dpi=200)

### Regions of stability 

In [ ]:
def coeff_arrays(tmats):
    Axx, Ayy, Axy = [], [], []
    for M in tmats:    
        Axx.append([M[0, 0]**2, M[0, 1]**2, 2*M[0, 0]*M[0, 1]])
        Ayy.append([M[2, 2]**2, M[2, 3]**2, 2*M[2, 2]*M[2, 3]])
        Axy.append([M[0, 0]*M[2, 2],  M[0, 1]*M[2, 2],  M[0, 0]*M[2, 3],  M[0, 1]*M[2, 3]])
    return np.array(Axx), np.array(Ayy), np.array(Axy)

In [ ]:
N = 4 # four wire-scanners
n_steps = 300
dmuxx = np.radians(np.linspace(0.01, 181, n_steps))
dmuyy = np.radians(np.linspace(0.01, 181, n_steps))

condition_numbers = np.zeros((n_steps, n_steps))
fail_rates = np.zeros((n_steps, n_steps))
for i, dmux in enumerate(tqdm(dmuxx)):
    for j, dmuy in enumerate(dmuyy):
        muxx = np.cumsum(np.full(N, dmux)) - dmux
        muyy = np.cumsum(np.full(N, dmuy)) - dmuy
        tmats = [phase_adv_matrix(mux, muy) for mux, muy in zip(muxx, muyy)]
        Axx, Ayy, Axy = coeff_arrays(tmats)
        condition_numbers[i, j] = cond(Axy) + cond(Axx) + cond(Ayy)

In [ ]:
fig, ax = plt.subplots()
X, Y = np.meshgrid(np.degrees(dmuxx), np.degrees(dmuyy))
Z = 2 / (1 + condition_numbers)
mesh = ax.pcolormesh(X, Y, Z, snap=True, cmap='binary_r', shading='auto')
cbar = fig.colorbar(mesh, ax=ax)
cbar.set_label(r'2 / (1 + C)')
ax.set_xlabel('Horizontal phase spacing [deg]')
ax.set_ylabel('Vertical phase spacing [deg]')
ax.set_title(r' Stability regions (evenly spaced WS; $\beta = 1$; $\alpha = 0$)');
plt.savefig('_output/stability_fodo.png', facecolor='white', dpi=300)

We want to create a picture like this for the RTBT wire-scanners. The three phase advances between the wire-scanners in the RTBT are not equal. They are also no located at points where the design $\beta$ function is maximum/minimum.

In [ ]:
def augmented_matrix(A, b):
    return np.hstack([A, b[:, np.newaxis]])

## Errors 

### Bounds on cross-plane moments

In [ ]:
def solve_corr_coeff_bounds(meas_index, **solver_kws):
    Axx, Ayy, Axy, bxx, byy, bxy = form_coeff_target_arrays(meas_index)
    sig_11, sig_22, sig_12 = opt.lsq_linear(Axx, bxx).x
    sig_33, sig_44, sig_34 = opt.lsq_linear(Ayy, byy).x  

    def cost_func(vec):
        return np.sum((np.matmul(Axy, vec) - bxy)**2)

    r_13_denom = np.sqrt(sig_11 * sig_33)
    r_23_denom = np.sqrt(sig_22 * sig_33)
    r_14_denom = np.sqrt(sig_11 * sig_44)
    r_24_denom = np.sqrt(sig_22 * sig_44)
    lb = [-r_13_denom, -r_23_denom, -r_14_denom, -r_24_denom]
    ub = [+r_13_denom, +r_23_denom, +r_14_denom, +r_24_denom]
    guess = 0.75 * np.array([lb[0], 0., 0., lb[3]])
    result = opt.least_squares(cost_func, guess, bounds=(lb, ub), **solver_kws)
    
    sig_13, sig_23, sig_14, sig_24 = result.x
    S = to_mat([sig_11, sig_22, sig_12, sig_33, sig_44, sig_34, sig_13, sig_23, sig_14, sig_24])
    return ba.emittances(S)

In [ ]:
# solver_kws = dict(max_nfev=1000)
# emittances = [solve_corr_coeff_bounds(meas_index, **solver_kws) for meas_index in trange(n_meas)]    
# plot_with_error_bars(emittances, eps_labels, correct=emittances_all_meas);

### Edwards-Teng

We can parameterize the covariance matrix as 

$$
\mathbf{\Sigma} = \mathbf{V} \, \mathbf{C} \, \mathbf{\Sigma}_n \, \mathbf{C}^T \, \mathbf{V}^T,
$$

with

$$
\mathbf{\Sigma}_n = 
\begin{bmatrix}
    \varepsilon_1 & 0 & 0 & 0 \\
    0 & \varepsilon_1 & 0 & 0 \\
    0 & 0 & \varepsilon_2 & 0 \\
    0 & 0 & 0 & \varepsilon_2
\end{bmatrix}, \\
$$

$$
\mathbf{C} = 
\begin{bmatrix}
    1 & 0 & a & b \\
    0 & 1 & c & d \\
    -d & b & 1 & 0 \\
    c & a & 0 & 1
\end{bmatrix}, \\
$$

$$
\mathbf{V} = 
\begin{bmatrix}
    \sqrt{\beta_x} & 0 & 0 & 0 \\
    -\frac{\alpha_x}{\sqrt{\beta_x}} & \frac{1}{\sqrt{\beta_x}} & 0 & 0 \\
    0 & 0 & \sqrt{\beta_y} & 0 \\
    0 & 0 & -\frac{\alpha_y}{\sqrt{\beta_y}} & \frac{1}{\sqrt{\beta_y}}
\end{bmatrix}. \\
$$

$\mathbf{C}$ is symplectic if $ad - bc = 0$, so we set $d = bc/a$. If $a = b = c = d = 0$, then we get the normal uncoupled matrix with $\varepsilon_{x,y} = \varepsilon_{1,2}$. 

The strategy is to fit the $x$-$x'$ and $y$-$y'$ planes first, then choose $\varepsilon_1$, $\varepsilon_2$, $a$, $b$, and $c$.

In [ ]:
def get_cov(eps_1, eps_2, alpha_x, alpha_y, beta_x, beta_y, a, b, c):
    E = np.diag([eps_1, eps_1, eps_2, eps_2])
    V = V_matrix_4x4_uncoupled(alpha_x, alpha_y, beta_x, beta_y)
    if a == 0:
        if b == 0 or c == 0:
            d = 0
        else:
            raise ValueError("a is zero but b * c is not zero.")
    else:
        d = b * c / a
    C = np.array([[1, 0, a, b], [0, 1, c, d], [-d, b, 1, 0], [c, -a, 0, 1]])
    return np.linalg.multi_dot([V, C, E, C.T, V.T])

In [ ]:
def solve_edwards_teng(meas_index, **solver_kws):
    Axx, Ayy, Axy, bxx, byy, bxy = form_coeff_target_arrays(meas_index)
    sig_11, sig_22, sig_12 = opt.lsq_linear(Axx, bxx).x
    sig_33, sig_44, sig_34 = opt.lsq_linear(Ayy, byy).x 
    S = np.zeros((4, 4))
    S[:2, :2] = [[sig_11, sig_12], [sig_12, sig_22]]
    S[2:, 2:] = [[sig_33, sig_34], [sig_34, sig_44]]
    eps_x, eps_y = ba.apparent_emittances(S)
    alpha_x, alpha_y, beta_x, beta_y = ba.twiss2D(S)

    def cost_func(vec):
        eps_1, eps_2, a, b, c = vec
        S = get_cov(eps_1, eps_2, alpha_x, alpha_y, beta_x, beta_y, a, b, c)
        vec_xy = np.array([S[0, 2], S[1, 2], S[0, 3], S[1, 3]])
        cost = np.sum((np.matmul(Axy, vec_xy) - bxy)**2)
        f = 1.0
        cost += f * (S[0, 0] - sig_11)**2
        cost += f * (S[0, 1] - sig_12)**2
        cost += f * (S[1, 1] - sig_22)**2
        cost += f * (S[2, 2] - sig_33)**2
        cost += f * (S[2, 3] - sig_34)**2
        cost += f * (S[3, 3] - sig_44)**2
        return cost

    lb = [0., 0., -np.inf, -np.inf, -np.inf]
    ub = np.inf
    guess = [0.75 * eps_x, 0.75 * eps_y, 0.5, 0.5, -0.5]
    result = opt.minimize(cost_func, guess, bounds=opt.Bounds(lb, ub), **solver_kws)
    eps_1, eps_2, a, b, c = result.x
    S = get_cov(eps_1, eps_2, alpha_x, alpha_y, beta_x, beta_y, a, b, c)
    S[:2, :2] = [[sig_11, sig_12], [sig_12, sig_22]]
    S[2:, 2:] = [[sig_33, sig_34], [sig_34, sig_44]]
    return ba.emittances(S)

In [ ]:
# solver_kws = dict(method='trust-constr', options=dict(verbose=0, xtol=1e-15, maxiter=1000))
# emittances = [solve_edwards_teng(meas_index, **solver_kws) for meas_index in trange(n_meas)]
# plot_with_error_bars(emittances, eps_labels, correct=emittances_all_meas);

### Cholesky decomposition 

In [ ]:
def inverse_cholesky(x):
    L = np.array([[x[0], 0, 0, 0],
                  [x[1], x[2], 0, 0],
                  [x[3], x[4], x[5], 0],
                  [x[6], x[7], x[8], x[9]]])
    return np.matmul(L, L.T)

In [ ]:
def solve_cholesky(meas_index, **solver_kws):
    Axx, Ayy, Axy, bxx, byy, bxy = form_coeff_target_arrays(meas_index)
    sig_11, sig_22, sig_12 = opt.lsq_linear(Axx, bxx).x
    sig_33, sig_44, sig_34 = opt.lsq_linear(Ayy, byy).x  

    def cost_func(x):
        S = inverse_cholesky(x)
        vec_xx = np.array([S[0, 0], S[1, 1], S[0, 1]])
        vec_yy = np.array([S[2, 2], S[3, 3], S[2, 3]])
        vec_xy = np.array([S[0, 2], S[1, 2], S[0, 3], S[1, 3]])
        cost = 0.
        cost += np.sum((np.matmul(Axx, vec_xx) - bxx)**2)
        cost += np.sum((np.matmul(Ayy, vec_yy) - byy)**2)
        cost += np.sum((np.matmul(Axy, vec_xy) - bxy)**2)
        return cost

    lb = -np.inf
    ub = +np.inf
    guess = np.ones(10)
#     guess = np.random.uniform(-10, 10, size=10)
    result = opt.minimize(cost_func, guess, bounds=opt.Bounds(lb, ub), **solver_kws)
    S = inverse_cholesky(result.x)
    return ba.emittances(S)

In [ ]:
# solver_kws = dict(method='trust-constr', options=dict(verbose=1, maxiter=2000))
# emittances = [solve_cholesky(meas_index, **solver_kws) for meas_index in trange(n_meas)]
# plot_with_error_bars(emittances, eps_labels, correct=emittances_all_meas);

In [ ]:
def form_coeff_target_arrays(meas_index):
    Axx, Ayy, Axy = [], [], []
    bxx, byy, bxy = [], [], []
    for ws_id in ws_ids:
        M = tmats_dict[ws_id][meas_index]
        sig_xx, sig_yy, sig_xy = moments_dict[ws_id][meas_index]
        Axx.append([M[0, 0]**2, M[0, 1]**2, 2*M[0, 0]*M[0, 1]])
        Ayy.append([M[2, 2]**2, M[2, 3]**2, 2*M[2, 2]*M[2, 3]])
        Axy.append([M[0, 0]*M[2, 2],  M[0, 1]*M[2, 2],  M[0, 0]*M[2, 3],  M[0, 1]*M[2, 3]])
        bxx.append(sig_xx)
        byy.append(sig_yy)
        bxy.append(sig_xy) 
    Axx, Ayy, Axy = np.array(Axx), np.array(Ayy), np.array(Axy)
    bxx, byy, bxy = np.array(bxx), np.array(byy), np.array(bxy)
    return Axx, Ayy, Axy, bxx, byy, bxy